In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import config


from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.specs import ArraySpec, BoundedArraySpec
from tf_agents.trajectories import time_step as ts
from sklearn.preprocessing import StandardScaler

In [17]:
# NEED TO CHANGE STOCKS VARIABLE
# NEED TO CHANGE NUM_OF_FEAT VARIABLE
# NEED TO CHANGE EPISODE_LENGTH VARIABLE
# NEED TO CHANGE FILEPATH VARIABLE
# NEED TO CHANGE DF.COL[1:7] IN GET OBS
# NEED TO CHANGE SCALED_COL 
# NEED TO CHANGE OBS_COL
tf.compat.v1.enable_v2_behavior()

class PortfolioEnv(py_environment.PyEnvironment):
    def __init__(self):
        # Action: Portfolio ratio in stocks and cash
        self._action_spec = BoundedArraySpec(shape=(len(stocks)+1,), 
                                                       minimum=0, 
                                                       maximum=1, 
                                                       dtype=np.float32, 
                                                       name='action')
        self._observation_spec = ArraySpec(shape=(num_of_feat,),
                                                     dtype=np.float32)
        self.reset()
        self._episode_ended = False
    
    def action_spec(self):
        return self._action_spec
    
    def observation_spec(self):
        return self._observation_spec

    def _reset(self):
        # Start new episode
        self._eposode_ended = False
        self.step_index = 0
        self.return_mem = pd.DataFrame(columns=[stock+'_Close' for stock in stocks])

        # Set timeframe: 1D
        self.timeframe = pd.Timedelta(1, unit='d')
        
        # Set starting capital and value
        self.capital = 1000
        self.cash = self.capital
        self.portfolio_value = self.capital

        # Store previous prices to calculate on days market is closed
        self.prev_price_share = {'JNJ': 0, 'KO': 0, 'T': 0}
        self.prev_price_cash = {'JNJ': 0, 'KO': 0, 'T': 0}

        # Initiate ratio array
        self.ratio = np.zeros((len(stocks)+1))
        self.ratio[0] = 1 # Cash ratio

        # Load data file:
        self.df = pd.read_csv(filepath,index_col=0).reset_index(drop=True)
        self.df['Date'] = pd.to_datetime(self.df['Date'])

        # Initiate Standard Scaler
        self.standard_scaler = StandardScaler()
        self.standard_scaler.fit(self.df[SCALED_COL].values)

        # Initiate reward
        self.step_reward = 0

        # Randomly slicing an episode out of the raw data for training
        self.df_max_index = self.df.shape[0]
        start_point = (np.random.choice(np.arange(3,self.df_max_index - episode_length))//3) *3 
        end_point = start_point + episode_length//3 *3
        self.df = self.df.loc[start_point:end_point+2].reset_index(drop=True)

        # First observation
        self.init_time = self.df.loc[0, 'Date']
        self.current_time = self.init_time
        self.df_time_slice = self.df[self.df['Date'] == self.current_time]

        # Calculate num of share for each stock
        self.current_stock_num_distribution = self.calculate_actual_shares_from_money_split()

        # Calculate portfolio value
        self.previous_value = self.portfolio_value
        self.current_stock_money_distribution, self.portfolio_value  = self.calculate_money_from_num_stocks()
        
        # State/Observation info
        self.info_ =  {"money_split":self.ratio,"share_num":self.current_stock_num_distribution,
                  "value":self.portfolio_value,"time":self.current_time,
                  "reward":self.step_reward,
                  "raw_output":self.get_observations_unscaled(),
                  "scaled_output":self.get_observations()}
        
        self._state = self.info_["scaled_output"][OBS_COL].values.flatten()
        
        self.info_["state"] = self._state
        reward = self.info_["reward"]
        

        return ts.restart(self._state)

    def _step(self, action):
        if self._episode_ended:
            self.reset()
        
        if sum(action) <= 0.003:
            self.ratio = [1/len(action)]
        else:
            self.ratio = [a/sum(action) for a in action]
            print(self.ratio)
        

        self.current_stock_num_distribution = self.calculate_actual_shares_from_money_split()
        self.step_time()
        self.step_index +=1

        # Step info
        self.info_ =  {"money_split":self.ratio,"share_num":self.current_stock_num_distribution,
                  "value":self.portfolio_value,"time":self.current_time,
                  "reward":self.step_reward,
                  "raw_output":self.get_observations_unscaled(),
                  "scaled_output":self.get_observations()}
        
        self._state = self.info_["scaled_output"][OBS_COL].values.flatten()
        self.info_["state"] = self._state
        reward = self.info_["reward"]

        self._episode_ended = True if self.step_index == episode_length//3 else False

        if self._episode_ended:
            reward = 0
            return ts.termination(self._state, reward)
        else:
            try:
                return ts.transition(self._state, reward=reward, discount=1)
            except Exception as e:
                print(e)
                print(self._state)
                print(reward)
                print(action)
                print(self.index)
                print(self.df_time_slice)
        
    def step_time(self):
        # Update step values
        self.current_time += self.timeframe
        self.df_time_slice = self.df[self.df['Date'] == self.current_time]
        self.previous_value = self.portfolio_value
        self.current_stock_money_distribution, self.portfolio_value = self.calculate_money_from_num_stocks()
        self.ratio = self.normalize_money_dist()
        self.step_reward = self.portfolio_value - self.previous_value
        
    def calculate_actual_shares_from_money_split(self):
        price_dict = self.df_time_slice[['Ticker', 'Open']].set_index('Ticker').to_dict()['Open']
        share_num = []

        for idx, stock in enumerate(stocks):
            if stock in price_dict:
                share_num.append(self.ratio[idx+1] * self.portfolio_value // price_dict[stock])
            else:
                share_num.append(self.ratio[idx+1] * self.portfolio_value // self.prev_price_share[stock])
            
        for stock in price_dict:
            self.prev_price_share[stock] = price_dict[stock]
        
        self.cash = self.ratio[0] * self.portfolio_value

        return share_num
    
    def calculate_money_from_num_stocks(self):
        price_dict = self.df_time_slice[['Ticker', 'Open']].set_index('Ticker').to_dict()['Open']
        distribution = []
        for idx, stock in enumerate(stocks):
            if stock in price_dict:
                distribution.append(self.current_stock_num_distribution[idx] * price_dict[stock])
            else:
                distribution.append(self.current_stock_num_distribution[idx] * self.prev_price_cash[stock])
                
        for stock in price_dict:
            self.prev_price_cash[stock] = price_dict[stock]


        return distribution, sum(distribution)
    
    def normalize_money_dist(self):
        normalized = []

        for idx, stock in enumerate(self.current_stock_money_distribution):
            normalized.append(stock/self.portfolio_value)
        
        return normalized
    
    def get_observations_unscaled(self):
        obs = pd.DataFrame()
        for i, frame in self.df_time_slice.groupby('Ticker'):
            temp_df = pd.DataFrame(frame[self.df.columns[1:7]].values)
            temp_df.columns = [i+"_"+c for c in self.df.columns[1:7]]
            if obs.empty:
                obs = temp_df
            else:
                obs = obs.merge(temp_df,right_index=True,left_index=True,how='inner')
            
        
        print(self.return_mem)
        print(obs)

        self.return_mem = pd.concat([self.return_mem, obs[[stock+'_Close' for stock in stocks]]], ignore_index=True)
        
        return obs
    
    def get_observations(self):
        obs = pd.DataFrame()
        for i,grp in self.df_time_slice.groupby("Ticker"):
            tempdf = pd.DataFrame(self.standard_scaler.transform(grp[SCALED_COL].values))
            tempdf.columns = [i+"_"+c for c in SCALED_COL]
            if obs.empty:
                obs = tempdf
            else:
                obs = obs.merge(tempdf,right_index=True,left_index=True,how='inner')
        
        return obs

            

In [6]:
class Environment(py_environment.PyEnvironment):
    def __init__(self):
        self._action_spec = BoundedArraySpec((len(config.TICKERS)+1,), 
                                             dtype=np.float64, 
                                             minimum=0, 
                                             maximum=1, 
                                             name='action')
        
        self._observation_spec = ArraySpec(shape=(len(config.OBS_COL),), 
                                           dtype=np.float64, 
                                           name='observation')
        self.reset()
        self._episode_ended = False
    
    def action_spec(self):
        return self._action_spec

    def observation_spec(self):
        return self._observation_spec
    
    def _reset(self):
        self.return_mem = pd.DataFrame(columns=[t+"_close" for t in config.TICKERS])
        self._episode_ended = False
        self.step_index = 0
        self.timeframe = pd.Timedelta(1,unit='d')
        self.init_cash = 1000
        self.current_cash = self.init_cash
        self.portfolio_value = self.init_cash

        # self.previous_price = {}
        self.prev_price_share = {}
        self.prev_price_cash = {}

        self.ratio = np.zeros((len(config.TICKERS)+1))
        self.ratio[0] = 1

        self.df = pd.read_csv(config.FILEPATH,index_col=0).reset_index(drop=True)
        self.scaler = StandardScaler()
        
        self.df["Date"] = pd.to_datetime(self.df["Date"])
        self.scaler.fit(self.df[config.EN_COLS].values)

        self.max_index = self.df.shape[0]
        start_point = (np.random.choice(np.arange(3,self.max_index - config.EPISODE_LENGTH))//3) *3
        end_point = start_point + config.EPISODE_LENGTH//3 *3
        self.df = self.df.loc[start_point:end_point+2].reset_index(drop=True)


        self.init_time = self.df.loc[0,"Date"]
        self.current_time = self.init_time
        self.df_time_slice = self.df[self.df['Date'] == self.init_time]

        self.current_share_distribution = self.calculate_actual_shares_from_money_split()
        self.previous_value = self.portfolio_value
        self.current_money_distribution,self.portfolio_value  = self.calculate_money_from_num_stocks()

        
        self.step_reward = 0
        
        self._state = self.get_observations()[config.OBS_COL].values.flatten()
        info_ =  {"state":self._state,\
                "money_split":self.ratio,"share_num":self.current_share_distribution,\
                "value":self.portfolio_value,"time":self.current_time,\
                "reward":self.step_reward,\
                # "raw_output":self.get_observations_unscaled(),
                "scaled_output":self.get_observations()}
        reward = info_["reward"]
        # self._episode_ended = True if self.index==config.EPISODE_LENGTH//3 else False
        

        return ts.restart(self._state)
    
    def _step(self, action):
 
        if self._episode_ended:
            return self.reset()
        if sum(action)<=1e-3:
            self.ratio = [1/len(action) for a in action]
        else:
            self.ratio = [a/sum(action) for a in action]

        self.current_stock_num_distribution = self.calculate_actual_shares_from_money_split()
        self.step_time()
        self.step_index +=1

        info_ =  {"state":"state",\
                    "money_split":self.ratio,"share_num":self.current_stock_num_distribution,\
                    "value":self.portfolio_value,"time":self.current_time,\
                    "reward":self.step_reward,\
                    "scaled_output":self.get_observations()}
 
        self._state = info_["scaled_output"][config.OBS_COL].values.flatten()
        reward = info_["reward"]
        self._episode_ended = True if self.step_index==config.EPISODE_LENGTH//3 else False

        if self._episode_ended:
            reward = 0
            return ts.termination(self._state , reward)
        else:
            try:
                return ts.transition(
                    self._state, reward=reward, discount=1)
            except Exception as e:
                print(self._state)
                print(reward)
                print(action)
                print(self.step_index)
                print(self.df_time_slice)
                print(self.current_time)
                print(self.ratio)
                print(e)
                raise ValueError
        
    def step_time(self):
        self.current_time += self.timeframe
        self.df_time_slice = self.df[self.df['Date'] == self.current_time]
        self.previous_value = self.portfolio_value
        self.current_money_distribution,self.portfolio_value  = self.calculate_money_from_num_stocks()
        self.ratio = self.normalize_money_dist()
        self.step_reward = self.portfolio_value - self.previous_value
    
    def get_observations(self):
        obs = pd.DataFrame()
        if self.df_time_slice.empty == False:
            for i,grp in self.df_time_slice.groupby("Ticker"):
                tempdf = pd.DataFrame(self.scaler.transform(grp[config.EN_COLS].values))
                tempdf.columns = [i+"_"+c for c in config.EN_COLS]
                if obs.empty:
                    obs = tempdf
                else:
                    obs = obs.merge(tempdf,right_index=True,left_index=True,how='inner')

        return obs
    
    def get_observations_unscaled(self):
        obs = pd.DataFrame()
        if self.df_time_slice.empty == False:
            for i,grp in self.df_time_slice.groupby("Ticker"):
                tempdf = pd.DataFrame(grp[config.COLS].values)
                tempdf.columns = [i+"_"+c for c in config.COLS]
                if obs.empty:
                    obs = tempdf
                else:
                    obs = obs.merge(tempdf,right_index=True,left_index=True,how='inner')
            
            self.memory_return = pd.concat([self.memory_return,obs[[t+"_Close" for t in config.TICKERS]]],ignore_index=True)
        
        return obs

    def calculate_actual_shares_from_money_split(self):
        price_dict = self.df_time_slice[["Ticker","Open"]]\
                        .set_index("Ticker").to_dict()["Open"]
        
        num_shares = []
        if len(price_dict) != 0:
            for i,c in enumerate(config.TICKERS):
                if c in price_dict:
                    num_shares.append(self.ratio[i+1]*self.portfolio_value//price_dict[c] )
                else:
                    num_shares.append(self.ratio[i+1]*self.portfolio_value//self.prev_price_share[c] )
                
            self.current_cash = self.ratio[0]*self.portfolio_value
            for c in price_dict:
                self.prev_price_share[c] = price_dict[c]
        
            return num_shares
        else:
            return self.current_share_distribution

    def calculate_money_from_num_stocks(self):
        money_dist = []
        money_dist.append(self.current_cash)
        price_dict = self.df_time_slice[["Ticker","Open"]]\
                        .set_index("Ticker").to_dict()["Open"]
        
        if len(price_dict) != 0:
            for i,c in enumerate(config.TICKERS):
                if c in price_dict:
                    money_dist.append(self.current_share_distribution[i]*price_dict[c])
                else:
                    money_dist.append(self.current_share_distribution[i]*self.prev_price_cash[c])
            
            for c in price_dict:
                self.prev_price_cash[c] = price_dict[c]
            return money_dist,sum(money_dist)
        else:
            return self.current_money_distribution, self.portfolio_value
        
    def normalize_money_dist(self):
        normal = []
        
        for i,c in enumerate(self.current_money_distribution):
            normal.append(c/self.portfolio_value)
        return normal

In [7]:
env = Environment()

KeyError: 'coin'

1